In [1]:
!pip install scrapy

In [1]:
import scrapy
from scrapy.crawler import CrawlerProcess
from urllib.parse import urljoin

class GravataiNews(scrapy.Spider):
    name = 'gr-news-spdier'
    start_urls = ['http://www.gravatai.ifsul.edu.br/ultimas-noticias']

    def parse(self, response):
        ONE_NEWS_SELECTOR = '.item-page'
        NEWS_ITEM_SELECTOR = '.tileItem'
        NEXT_SELECTOR = '.pagination-next a::attr("href")'
        NEWS_LINKS_SELECTOR = '.tileHeadline a::attr("href")'
        
        # Acessa uma noticia da lista
        for news in response.css(NEWS_ITEM_SELECTOR):
            yield scrapy.Request(
                urljoin('http://www.gravatai.ifsul.edu.br',news.css(NEWS_LINKS_SELECTOR).extract_first())
            )

        # proxima lista de noticias
        next_page = response.css(NEXT_SELECTOR).extract_first()
        if next_page:
            yield scrapy.Request(
                response.urljoin(next_page),
            )
        
        # extrai o texto de uma pagina de noticia
        news = response.css(ONE_NEWS_SELECTOR)
        if news:
            texto = news.css('*::text').getall()
            if len(texto) > 0:
                yield {
                    'noticia': ' '.join(texto).strip(),
                    'url': response.url
                }

            
process = CrawlerProcess(
    settings={
        "FEEDS": {
            "noticias.json": {"format": "json"},
        },
    }
)

process.crawl(GravataiNews)
process.start()

2023-09-18 15:53:35 [scrapy.utils.log] INFO: Scrapy 2.8.0 started (bot: scrapybot)
2023-09-18 15:53:35 [scrapy.utils.log] INFO: Versions: lxml 4.9.2.0, libxml2 2.10.3, cssselect 1.1.0, parsel 1.6.0, w3lib 1.21.0, Twisted 22.10.0, Python 3.11.3 (main, Apr 19 2023, 18:49:55) [Clang 14.0.6 ], pyOpenSSL 23.0.0 (OpenSSL 1.1.1u  30 May 2023), cryptography 39.0.1, Platform macOS-13.5.2-arm64-arm-64bit
2023-09-18 15:53:35 [scrapy.crawler] INFO: Overridden settings:
{}
2023-09-18 15:53:35 [py.warnings] WARNING: /Users/dias/anaconda3/lib/python3.11/site-packages/scrapy/utils/request.py:232: ScrapyDeprecationWarning: '2.6' is a deprecated value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting.

It is also the default value. In other words, it is normal to get this warning if you have not defined a value for the 'REQUEST_FINGERPRINTER_IMPLEMENTATION' setting. This is so for backward compatibility reasons, but it will change in a future version of Scrapy.

See the documentation of the 'REQUES